In [9]:
from abc import ABC, abstractmethod
from dataclasses import dataclass, fields, asdict
import gc
from tqdm import tqdm

import torch

from model.model import PageAccModel
from logfile_reader import read_pages, save_pages_accs, read_optimal_results, Page

In [11]:
BUFFER_SIZE = 32
TRAIN_PART = 0.7
RESULTS_DIR = "results/data/tpcc_32/"

In [12]:
pages = read_pages(RESULTS_DIR + "logfile")
train_size = int(len(pages) * TRAIN_PART)
test_size = len(pages) - train_size
train_pages = pages[:train_size]
test_pages = pages[train_size:]

In [4]:
class IPageVictimStrategy(ABC):
    @abstractmethod
    def forward(page: Page, buffer: list[Page]):
        pass

In [17]:
def get_metrics(pages: list[Page], evict_strategy: IPageVictimStrategy):
    buffer = [Page(-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0)] * BUFFER_SIZE

    hits = 0
    hit_rates = []
    pbar = tqdm(range(len(pages)))
    for i in pbar:
        page = pages[i]
        victim = evict_strategy.forward(page, buffer)
        if victim < 0:
            hits += 1
        else:
            # buffer[victim] = deepcopy(page)
            buffer[victim] = page
            buffer[victim].hit = victim
        
        hit_rates.append(hits / (i + 1))

        if i % 10000 == 0:
            pbar.set_postfix_str(f"hit_rate={hits / (i + 1)}")
    
    return hit_rates

In [6]:
class ModelVictimStrategy(IPageVictimStrategy):
    def __init__(self, model: PageAccModel):
        self._model = model
        self._h = None
        self._c = None

        self._model.eval()

    def forward(self, page: Page, buffer: list[Page]):
        page_tensor = torch.Tensor(list(asdict(page).values()))
        page_in_buffer = next(filter(lambda x: x[1].buffer == page.buffer, enumerate(buffer)), None)
        if page_in_buffer is not None:
            page_tensor[-1] = int(page_in_buffer[0])
        page_tensor = page_tensor.view(1, len(page_tensor))

        buffer_tensor = torch.Tensor([value for obj in buffer for value in asdict(obj).values()])
        buffer_tensor = buffer_tensor.view(1, len(buffer_tensor))

        with torch.no_grad():
            out, self._h, self._c = self._model.forward(page_tensor, buffer_tensor, self._h, self._c)

        self._h.detach()
        self._c.detach()

        if page_in_buffer is not None:
            return -1

        return torch.argmax(out, dim=1)[0]

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PageAccModel(11, 512, 1024, 32)
model.load_state_dict(torch.load(RESULTS_DIR + "model.pth", map_location=device, weights_only=True))
model.eval()

model_evict_strategy = ModelVictimStrategy(model)

In [18]:
hit_rate = get_metrics(test_pages, model_evict_strategy)

  1%|          | 3410/307221 [00:41<1:02:16, 81.31it/s, hit_rate=0.0]


KeyboardInterrupt: 